In [30]:
import os
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_teddynote.tools.tavily import TavilySearch

In [32]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tavily = TavilySearch()

In [33]:
# 그래프 상태 정의
class GraphState(TypedDict):
    company_name: Annotated[str, "기업 이름"]
    domain: Annotated[str, "기업 도메인"]
    founder_name: Annotated[str, "창업자 이름"]
    founder_role: Annotated[str, "창업자 역할"]
    profile_info: Annotated[str, "창업자 프로필 정보"]
    reputation_info: Annotated[str, "창업자 평판 정보"]
    sentiment_analysis: Annotated[str, "평판 긍정/부정 분석"]
    final_summary: Annotated[str, "최종 요약"]
    messages: Annotated[list[BaseMessage], "메시지"]
    relevance: Annotated[bool, "관련성"]


In [34]:
# 1. 창업자 식별 에이전트
def founder_identifier(state: GraphState) -> GraphState:
    print(f"🔍 창업자 식별 에이전트: {state['company_name']} 창업자 식별 중...")
    
    search_query = f"{state['company_name']} {state['domain']} 스타트업 창업자 CEO 대표 설립자"
    
    search_results = tavily.search(
        query=search_query,
        topic="general",
        days=60,
        max_results=5,
        format_output=True,
    )
    
    # 검색 결과를 LLM에 전달하여 창업자 이름과 역할 추출
    extraction_prompt = f"""
    다음은 {state['company_name']} 기업에 관한 정보입니다:
    
    {search_results}
    
    위 정보를 바탕으로 이 기업의 창업자(설립자) 또는 현재 CEO의 이름과 역할을 추출해주세요.
    여러 명이 있다면, 핵심 인물 한 명만 선택해주세요.
    
    다음 형식으로 응답해주세요:
    창업자 이름: [이름]
    창업자 역할: [역할 (예: CEO, 공동창업자, CTO 등)]
    """
    
    extraction_response = llm.invoke(extraction_prompt)
    extraction_content = extraction_response.content
    
    # 응답에서 창업자 이름과 역할 추출
    founder_name = ""
    founder_role = ""
    
    for line in extraction_content.split('\n'):
        if "창업자 이름:" in line or "이름:" in line:
            founder_name = line.split(':')[1].strip()
        elif "창업자 역할:" in line or "역할:" in line:
            founder_role = line.split(':')[1].strip()
    
    print(f"✓ 식별된 창업자: {founder_name} ({founder_role})")
    
    return GraphState(
        founder_name=founder_name,
        founder_role=founder_role
    )

In [35]:
# 2. 창업자 정보 수집 에이전트
def profile_collector(state: GraphState) -> GraphState:
    print(f"🔍 정보 수집 에이전트: {state['founder_name']} 정보 수집 중...")
    
    search_query = f"{state['founder_name']} {state['company_name']} {state['founder_role']} 경력 학력 성과 이력"
    
    search_results = tavily.search(
        query=search_query,
        topic="general",
        days=60,
        max_results=5,
        format_output=True,
    )
    
    # 검색 결과 포맷팅
    profile_info = f"## {state['founder_name']} ({state['founder_role']}) 프로필 정보\n\n"
    profile_info += "\n\n".join(search_results)
    
    print(f"✅ {state['founder_name']}의 프로필 정보 수집 완료")
    
    return GraphState(profile_info=profile_info)

In [36]:
# 3. 평판 분석 에이전트
def reputation_analyzer(state: GraphState) -> GraphState:
    print(f"🔍 평판 분석 에이전트: {state['founder_name']} 평판 분석 중...")
    
    search_query = f"{state['founder_name']} {state['company_name']} 평판 인터뷰 SNS 미디어 리뷰"
    
    search_results = tavily.search(
        query=search_query,
        topic="news",
        days=180,  # 더 넓은 기간 설정
        max_results=5,
        format_output=True,
    )
    
    # 검색 결과 포맷팅
    reputation_info = f"## {state['founder_name']} ({state['company_name']}) 미디어 및 SNS 평판\n\n"
    reputation_info += "\n\n".join(search_results)
    
    # 감성 분석 수행
    sentiment_prompt = f"""
    다음은 {state['company_name']}의 {state['founder_name']} {state['founder_role']}에 관한 미디어 및 SNS 정보입니다. 
    이 내용을 분석하여 긍정적/부정적 평판을 판단해주세요.
    
    정보:
    {reputation_info}
    
    다음 형식으로 분석해주세요:
    1. 감성 점수: (0-100 사이, 0이 매우 부정적, 100이 매우 긍정적)
    2. 주요 긍정적 언급:
    3. 주요 부정적 언급:
    4. 전반적인 평판 판단:
    5. 투자 관점에서의 시사점:
    """
    
    sentiment_response = llm.invoke(sentiment_prompt)
    sentiment_analysis = sentiment_response.content
    
    print(f"✅ {state['founder_name']}의 평판 분석 완료")
    
    return GraphState(
        reputation_info=reputation_info,
        sentiment_analysis=sentiment_analysis
    )

In [37]:
# 4. 요약 에이전트
def summary_generator(state: GraphState) -> GraphState:
    print(f"📝 요약 에이전트: {state['company_name']}의 {state['founder_name']} 최종 요약 생성 중...")
    
    summary_prompt = f"""
    다음은 {state['company_name']} ({state['domain']}) 기업의 창업자/대표 {state['founder_name']} ({state['founder_role']})에 관한 정보입니다. 
    이 정보를 바탕으로 AI 스타트업 투자 가능성 평가를 위한 요약 보고서를 작성해주세요.
    
    ## 기본 프로필 정보
    {state['profile_info']}
    
    ## 평판 정보
    {state['reputation_info']}
    
    ## 감성 분석 결과
    {state['sentiment_analysis']}
    
    다음 형식으로 요약해주세요:
    1. 창업자 기본 정보 (이름, 역할, 기업명)
    2. 창업자 이력 요약 (학력, 경력, 성과, 현재 직책)
    3. 평판 분석 요약 (미디어/SNS에서의 이미지)
    4. 강점 및 약점
    5. 투자 관점에서의 시사점 (창업자 역량이 기업 성장에 미치는 영향)
    """
    
    summary_response = llm.invoke(summary_prompt)
    final_summary = summary_response.content
    
    print(f"✅ {state['company_name']}의 {state['founder_name']} 최종 요약 생성 완료")
    
    # 메시지 생성
    messages = [
        HumanMessage(content=f"{state['company_name']} ({state['domain']}) 기업의 {state['founder_name']} {state['founder_role']}에 대한 정보 분석 결과입니다."),
        AIMessage(content=final_summary)
    ]
    
    return GraphState(
        final_summary=final_summary,
        messages=messages
    )


In [38]:
# 창업자 식별 관련성 체크 함수
def founder_relevance_check(state: GraphState) -> GraphState:
    print("⚖️ 창업자 식별 관련성 체크 중...")
    
    # 창업자 정보가 비어있거나 불충분한지 체크
    if not state['founder_name'] or state['founder_name'].strip() == "":
        print("❌ 창업자 정보를 찾을 수 없습니다.")
        return GraphState(relevance=False)
    
    # 식별된 창업자와 기업의 관련성 체크
    relevance_prompt = f"""
    질문: {state['founder_name']}이(가) {state['company_name']} ({state['domain']}) 기업의 창업자/CEO/주요 임원인가요?
    
    창업자 이름: {state['founder_name']}
    창업자 역할: {state['founder_role']}
    기업명: {state['company_name']}
    
    이 정보가 일치하는지 판단해주세요. "yes" 또는 "no"로만 대답해주세요.
    """
    
    relevance_response = llm.invoke(relevance_prompt)
    is_relevant = "yes" in relevance_response.content.lower()
    
    print(f"창업자-기업 관련성 판단: {is_relevant}")
    
    return GraphState(relevance=is_relevant)



In [39]:
# 프로필 정보 관련성 체크 함수
def profile_relevance_check(state: GraphState) -> GraphState:
    print("⚖️ 프로필 정보 관련성 체크 중...")
    
    # 프로필 정보와 창업자 이름의 관련성 체크
    relevance_prompt = f"""
    다음은 {state['company_name']} 기업의 {state['founder_name']} ({state['founder_role']})에 관한 검색 결과입니다:
    
    {state['profile_info']}
    
    이 정보가 실제로 해당 인물에 관한 것인지 판단해주세요.
    관련이 있다면 "yes", 없다면 "no"로만 대답해주세요.
    """
    
    relevance_response = llm.invoke(relevance_prompt)
    is_relevant = "yes" in relevance_response.content.lower()
    
    print(f"프로필 정보 관련성 판단: {is_relevant}")
    
    return GraphState(relevance=is_relevant)

In [40]:
# 조건부 라우팅 함수
def is_relevant(state: GraphState) -> str:
    if state["relevance"]:
        return "yes"
    else:
        return "no"


In [41]:
# 워크플로우 그래프 정의
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("founder_identifier", founder_identifier)
workflow.add_node("founder_relevance_check", founder_relevance_check)
workflow.add_node("profile_collector", profile_collector)
workflow.add_node("profile_relevance_check", profile_relevance_check)
workflow.add_node("reputation_analyzer", reputation_analyzer)
workflow.add_node("summary_generator", summary_generator)

# 엣지 추가
workflow.add_edge(START, "founder_identifier")
workflow.add_edge("founder_identifier", "founder_relevance_check")
workflow.add_conditional_edges(
    "founder_relevance_check",
    is_relevant,
    {
        "yes": "profile_collector",
        "no": "founder_identifier"  # 창업자 식별 실패 시 다시 시도
    }
)
workflow.add_edge("profile_collector", "profile_relevance_check")
workflow.add_conditional_edges(
    "profile_relevance_check",
    is_relevant,
    {
        "yes": "reputation_analyzer",
        "no": "profile_collector"  # 프로필 정보 관련성 없으면 다시 검색
    }
)
workflow.add_edge("reputation_analyzer", "summary_generator")
workflow.add_edge("summary_generator", END)

# 그래프 컴파일
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [42]:
# 실행 함수
def analyze_startup_founder(company_name: str, domain: str):
    config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
    
    inputs = GraphState(
        company_name=company_name,
        domain=domain,
        founder_name="",
        founder_role="",
        profile_info="",
        reputation_info="",
        sentiment_analysis="",
        final_summary="",
        messages=[],
        relevance=False
    )

    final_result = app.invoke(inputs, config)
    return final_result


In [43]:
# 실행 예시
if __name__ == "__main__":
    # company_name = input("분석할 AI 스타트업 기업명을 입력하세요: ")
    # domain = input("기업 도메인 분야를 입력하세요(예: AI, 헬스케어, 핀테크 등): ")
    company_name = "올거나이즈"
    domain = "AI"
    
    print(f"\n{company_name} ({domain}) 기업의 창업자 분석을 시작합니다...\n")
    result = analyze_startup_founder(company_name, domain)

    print(type(result))
    
    # print("\n최종 분석 결과:")
    # if "final_summary" in result:
    #     print(result["final_summary"])
    # else:
    #     print("분석 결과가 없습니다. 다시 시도해 주세요.")




올거나이즈 (AI) 기업의 창업자 분석을 시작합니다...

🔍 창업자 식별 에이전트: 올거나이즈 창업자 식별 중...
✓ 식별된 창업자: 이창수 (CEO)
⚖️ 창업자 식별 관련성 체크 중...
창업자-기업 관련성 판단: True
🔍 정보 수집 에이전트: 이창수 정보 수집 중...
✅ 이창수의 프로필 정보 수집 완료
⚖️ 프로필 정보 관련성 체크 중...
프로필 정보 관련성 판단: True
🔍 평판 분석 에이전트: 이창수 평판 분석 중...
✅ 이창수의 평판 분석 완료
📝 요약 에이전트: 올거나이즈의 이창수 최종 요약 생성 중...
✅ 올거나이즈의 이창수 최종 요약 생성 완료
<class 'langgraph.pregel.io.AddableValuesDict'>


In [44]:
type(result)

langgraph.pregel.io.AddableValuesDict